### Case Study

Many strategies can be highly profitable depending on whether you are in an upward trending, downward trending or oscillating market. However, how can you tell what market you are in and in addition do this without an subjective bias?

Hidden Markov Models allow us to determine any number of states we like, given inputs such as returns and volatility (or any inputs of your choosing), to help ascertain what state - or regime - the market is in. This is part of a type of analysis known as Regime identification.

### Imports

In [1]:

import pandas as pd
import numpy as np

from pyhhmm.gaussian import GaussianHMM


import matplotlib.pyplot as plt
import fix_yahoo_finance
import yfinance

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/fix_yahoo_finance/__init__.py:0: DeprecationWarning: 

*** `fix_yahoo_finance` was renamed to `yfinance`. ***
Please install and use `yfinance` directly using `pip install yfinance -U`

More information: https://github.com/ranaroussi/yfinance



### Data Management

In [2]:
# Data Extraction
start_date = "2017-01-1"
end_date = "2022-06-1"
symbol = "SPY"

data = yfinance.download(tickers = "RSG.AX",start="2000-03-06",
                               interval = "1d", group_by = 'ticker', auto_adjust = True)



data = data[["Open", "High", "Low", "Close","Volume"]]


[*********************100%***********************]  1 of 1 completed


In [3]:
# Add Returns and Range
df = data.copy()
df["Returns"] = (df["Close"] / df["Close"].shift(1)) - 1
df["Range"] = (df["High"] / df["Low"]) - 1
df.dropna(inplace=True)
df.head()

,Open,High,Low,Close,Volume,Returns,Range
Date,,,,,,,
2000-03-07,1.029043,1.049625,1.000231,1.000231,67491,-0.027999,0.049383
2000-03-08,1.008462,1.029043,0.987881,0.987881,107376,-0.012346,0.041666
2000-03-09,0.987881,1.008462,0.946720,0.967301,202665,-0.020833,0.065217
2000-03-10,0.987881,1.008462,0.967301,1.008462,48220,0.042553,0.042553
2000-03-13,0.959068,1.029043,0.959068,1.029043,72981,0.020408,0.072961


In [4]:
# Structure Data
X_train = df[["Returns", "Range"]]
X_train.head()

,Returns,Range
Date,,
2000-03-07,-0.027999,0.049383
2000-03-08,-0.012346,0.041666
2000-03-09,-0.020833,0.065217
2000-03-10,0.042553,0.042553
2000-03-13,0.020408,0.072961


### HMM Learning

In [ ]:
# Train Model
model = GaussianHMM(n_states=4, covariance_type='full', n_emissions=2)
model.train([np.array(X_train.values)])

/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
# Check Results
hidden_states = model.predict([X_train.values])[0]
print(hidden_states[:40])
len(hidden_states)

In [ ]:
# Regime state means for each feature
model.means

In [ ]:
# Regime state covars for each feature
model.covars

### Data Visualization

In [ ]:
# Structure the prices for plotting
i = 0
labels_0 = []
labels_1 = []
labels_2 = []
labels_3 = []
prices = df["Close"].values.astype(float)
print("Correct number of rows: ", len(prices) == len(hidden_states))
for s in hidden_states:
    if s == 0:
        labels_0.append(prices[i])
        labels_1.append(float('nan'))
        labels_2.append(float('nan'))
        labels_3.append(float('nan'))
    if s == 1:
        labels_0.append(float('nan'))
        labels_1.append(prices[i])
        labels_2.append(float('nan'))
        labels_3.append(float('nan'))
    if s == 2:
        labels_0.append(float('nan'))
        labels_1.append(float('nan'))
        labels_2.append(prices[i])
        labels_3.append(float('nan'))
    if s == 3:
        labels_0.append(float('nan'))
        labels_1.append(float('nan'))
        labels_2.append(float('nan'))
        labels_3.append(prices[i])
    i += 1

In [ ]:
# Plot Chart
fig = plt.figure(figsize= (18, 8))
plt.plot(labels_0, color="green")
plt.plot(labels_1, color="red")
plt.plot(labels_2, color="orange")
plt.plot(labels_3, color="black")
plt.show()

### Conclusion

Although work is still yet to be done in the following notebook, we can cleary see from the chart above, that the Hidden Markov Model has been able to identify market regimes based on returns and volatility behaviour.

### Useful Resources

HMM Colab Version with Backtest: https://colab.research.google.com/drive/12qzR8SrhfhQDBImKYQqUKdj6n60E9jNp?usp=sharing